In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simplified Self-attention

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [5]:
query = inputs[1] # the second input token embedding "journey"
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
# The above are the un-normalized attention scores, we have to normalize them for a better representation of the attention scores

In [10]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f'Attention weights: {attn_weights_2_tmp}' )
print(f'Sum: {attn_weights_2_tmp.sum()}')

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: 1.0000001192092896


In [11]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(f'Attention weights: {attn_weights_2_naive}')
print(f'Sum: {attn_weights_2_naive.sum()}')

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [12]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(f"Attention weights: {attn_weights_2}")
print(f"Sum: {attn_weights_2.sum()}")

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [13]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Generalizing the 3 step process to get context vectors for all input embeddings

In [15]:
inputs.shape

torch.Size([6, 3])

In [16]:
attn_scores = torch.empty(inputs.size(0), inputs.size(0))

for i, x_i in enumerate(inputs):
    query = x_i
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_j, query) 

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [17]:
# The above is literally just matrix multiplication, we'll use matmul instead of slow for loops

attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [18]:
# we have the attention scores, now step 2 normalize them
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [22]:
# we have the normalized attn weights now step 3 sum them up to get the context vector for each input
context_vecs = attn_weights @ inputs
context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

# Implementing self-attention with trainable weights

In self-attention we introduce trainable attention weights. There are three trainable weight matrics: W_q, W_k, W_v. These are used to project the input tokens into query, key, and value vectors.

In [23]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [24]:
# Creating the weight matrics
torch.manual_seed(123)

w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [25]:
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value
print(query_2)

tensor([0.4306, 1.4551])


In [26]:
keys = inputs @ w_key
values = inputs @ w_value
print(f"keys.shape: {keys.shape}")
print(f"values.shape: {values.shape}")

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [29]:
# Computing the attention scores w_22
keys_2 = keys[1]
attn_score_22 = torch.dot(query_2, keys_2)
print(f"un-normalized attention score {attn_score_22}")

un-normalized attention score 1.8523844480514526


In [30]:
# Generalize to all attention scores via matrix multiplication
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [31]:
# Now we want the attention weights, we do softmax, but scale the attention scores by dividing them by the square root of the embedding dimension
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [32]:
# The last step is multiplying each value vector with its attention weight, then summing them to get the context vector (weighted sum)
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


## Generalization to compute all context vectors

### Now we will use the created self attention class v1 and v2

In [47]:
torch.manual_seed(123)
from self_attention import SelfAttention_v1, SelfAttention_v2

sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [50]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0776,  0.0658],
        [-0.0772,  0.0662],
        [-0.0769,  0.0667],
        [-0.0780,  0.0652],
        [-0.0711,  0.0743],
        [-0.0810,  0.0613]], grad_fn=<MmBackward0>)


### Excercise 3.1 Comparing self-attention v1 and self-attention v2

In [52]:
torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.5350, -0.1070],
        [-0.5384, -0.1114],
        [-0.5383, -0.1114],
        [-0.5333, -0.1093],
        [-0.5349, -0.1099],
        [-0.5339, -0.1095]], grad_fn=<MmBackward0>)
